In [29]:
import os
import psycopg
from dotenv import load_dotenv

import pandas as pd
import mlflow
import numpy as np
from autofeat import AutoFeatClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, 
    f1_score, 
    precision_score, 
    recall_score,
    confusion_matrix,
    log_loss)

In [3]:
load_dotenv()

True

In [23]:
TABLE_NAME = 'users_churn'
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "Data_Exploration" # название эксперимента
RUN_NAME = "feature_generation" 

In [4]:
connection = {'sslmode' : 'require', 'target_session_attrs' : "read-write"}

postgres_credetials = {
    'dbname' : os.getenv('DB_DESTINATION_NAME'),
    'host' : os.getenv('DB_DESTINATION_HOST'),
    'port' : os.getenv('DB_DESTINATION_PORT'),
    'user' : os.getenv('DB_DESTINATION_USER'),
    'password' : os.getenv('DB_DESTINATION_PASSWORD')
}

connection.update(postgres_credetials)

In [7]:
with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f'SELECT * FROM {TABLE_NAME}')
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

        df = pd.DataFrame(data, columns=columns)


In [9]:
columns_without_datetime = df.select_dtypes(exclude='datetime').columns
df = df.dropna(subset=columns_without_datetime)

In [10]:
df.head()

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,Yes,No,Yes,No,No,No,Male,0,No,No,No,0
2,3,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,Yes,Yes,No,No,No,No,Male,0,No,No,No,1
4,5,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No,1
5,6,9305-CDSKC,2019-03-01,2019-11-01,Month-to-month,Yes,Electronic check,99.65,820.50,Fiber optic,No,No,Yes,No,Yes,Yes,Female,0,No,No,Yes,1
6,7,1452-KIOVK,2018-04-01,NaT,Month-to-month,Yes,Credit card (automatic),89.10,1949.40,Fiber optic,No,Yes,No,No,Yes,No,Male,0,No,Yes,Yes,0


In [13]:
X = df.drop(columns=['target', 'end_date'])
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [21]:
cat_features = [
    'paperless_billing',
    'payment_method',
    'internet_service',
    'online_security',
    'online_backup',
    'device_protection',
    'tech_support',
    'streaming_tv',
    'streaming_movies',
    'gender',
    'senior_citizen',
    'partner',
    'dependents',
    'multiple_lines',
]
num_features = ["monthly_charges", "total_charges"]

features = cat_features + num_features

transformations = ["log", "sqrt", "abs", "1/"]

afc = AutoFeatClassifier(
    categorical_cols=cat_features, 
    transformations=transformations, 
    feateng_steps=1, 
    n_jobs=-1)

X_train_features = afc.fit_transform(X_train[features], y_train)
X_test_features = afc.transform(X_test[features])


In [30]:
mlflow.set_tracking_uri(f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}')
mlflow.set_registry_uri(f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}')

In [27]:
artifact_path = 'afc'

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME) as run:
    run_id = run.info.run_id

    afc_info = mlflow.sklearn.log_model(afc, artifact_path=artifact_path) 

2025-02-07 14:52:41,051 INFO: Found credentials in environment variables.


In [28]:
display(experiment_id)
display(run_id)

'4'

'ff1d09222b5b4fc6988a26b88f7ef033'

In [31]:
model = LogisticRegression()

In [33]:
model.fit(X_train_features, y_train)

LogisticRegression()

In [34]:
prediction = model.predict(X_test_features)
proba = model.predict_proba(X_test_features)[:, 1]

In [35]:
metrics = {}

_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel()
auc = roc_auc_score(y_test, proba)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, prediction)

metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

In [36]:
metrics

{'err1': 0.10237849017580145,
 'err2': 0.18304033092037228,
 'auc': 0.8066974755138641,
 'precision': 0.6413043478260869,
 'recall': 0.5746753246753247,
 'f1': 0.6061643835616438,
 'logloss': 8.572947548600771}

In [38]:
EXPERIMENT_NAME = 'churn_fio'
RUN_NAME = 'feature_generation'
REGISTRY_MODEL_NAME = 'baseline_model_with_'

In [40]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

if experiment:
    experiment_id = experiment.experiment_id
else:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

In [41]:
with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME) as run:
    run_id = run.info.run_id
    mlflow.log_metrics(metrics)

    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME
    )

Successfully registered model 'baseline_model_with_'.
2025/02/07 15:08:08 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: baseline_model_with_, version 1
Created version '1' of model 'baseline_model_with_'.


In [43]:
run_id

'4ca696eab83b43c0830192c4f3cc88d6'